In [ ]:
# Neural Machine Translation using Attention mechanism using English to Marathi
#https://github.com/hlamba28/NMT-with-Attention-Mechanism/blob/master/NMT_Attention_Marathi.ipynb

In [1]:
# mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# import the libraries required
import tensorflow as tf


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time
import string



In [3]:
!pip install plotly
!pip install chart_studio
import plotly
import chart_studio.plotly as py
#import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go

     |████████████████████████████████| 71kB 8.4MB/s 


In [4]:
#path for the files
file_path = "/content/gdrive/My Drive/Colab Notebooks/mar.txt"

In [8]:
lines = open(file_path, encoding='UTF-8').read().strip().split('\n')
lines[5000:5010]

["Tom isn't naked.\tटॉम नागडा नाही आहे.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2236995 (CK) & #6741369 (sabretou)",
 "Tom isn't there.\tटॉम तिथे नाहीये.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2237026 (CK) & #6563837 (sabretou)",
 "Tom isn't vegan.\tटॉम व्हेगन नाहीये.\tCC-BY 2.0 (France) Attribution: tatoeba.org #8913949 (CK) & #8915092 (sabretou)",
 "Tom isn't weird.\tटॉम विचित्र नाहीये.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2237030 (CK) & #6563807 (sabretou)",
 'Tom kept trying.\tटॉमने प्रयत्न चालू ठेवले.\tCC-BY 2.0 (France) Attribution: tatoeba.org #6440615 (CK) & #6871730 (sabretou)',
 'Tom kept trying.\tटॉम प्रयत्न करत राहिला.\tCC-BY 2.0 (France) Attribution: tatoeba.org #6440615 (CK) & #6871732 (sabretou)',
 'Tom kicked Mary.\tटॉमने मेरीला लाथ मारली.\tCC-BY 2.0 (France) Attribution: tatoeba.org #1028439 (CK) & #4813726 (sabretou)',
 'Tom knows a lot.\tटॉमला भरपूर काही माहिती आहे.\tCC-BY 2.0 (France) Attribution: tatoeba.org #3171257 (CK) & #4010979 (sabretou)

In [9]:
len(lines)

40751

In [10]:
#!pip install modin[ray]

In [11]:
%%time
import pandas as pd
lines=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/mar.txt',sep="\t",header=None)
lines.head()

CPU times: user 102 ms, sys: 9.39 ms, total: 111 ms
Wall time: 125 ms


In [12]:
lines.shape

(40751, 3)

In [13]:
## Renamng the columns
lines.columns=['eng','mar','france']
lines.head()

,eng,mar,france
0,Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [14]:
# drop the france column
col_to_drop=['france']
lines=lines.drop(col_to_drop,axis=1)
lines.head()

,eng,mar
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!


In [15]:
exclude = set(string.punctuation) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) # Set of all digits

In [16]:
def preprocess_eng_sentence(sent):
    '''Function to preprocess English sentence'''
    sent = sent.lower() # lower casing
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [17]:
def preprocess_mar_sentence(sent):
    '''Function to preprocess Marathi sentence'''
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = re.sub("[२३०८१५७९४६]", "", sent) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [18]:
lines = open(file_path, encoding='UTF-8').read().strip().split('\n')
lines[5000:5010]
# Generate pairs of cleaned English and Marathi sentences
sent_pairs = []
for line in lines:
    sent_pair = []
    eng, mar,french = line.split('\t')
    eng = preprocess_eng_sentence(eng)
    sent_pair.append(eng)
    mar = preprocess_mar_sentence(mar)
    sent_pair.append(mar)
    sent_pairs.append(sent_pair)
sent_pairs[5000:5010]

[['<start> tom isnt naked <end>', '<start> टॉम नागडा नाही आहे <end>'],
 ['<start> tom isnt there <end>', '<start> टॉम तिथे नाहीये <end>'],
 ['<start> tom isnt vegan <end>', '<start> टॉम व्हेगन नाहीये <end>'],
 ['<start> tom isnt weird <end>', '<start> टॉम विचित्र नाहीये <end>'],
 ['<start> tom kept trying <end>', '<start> टॉमने प्रयत्न चालू ठेवले <end>'],
 ['<start> tom kept trying <end>', '<start> टॉम प्रयत्न करत राहिला <end>'],
 ['<start> tom kicked mary <end>', '<start> टॉमने मेरीला लाथ मारली <end>'],
 ['<start> tom knows a lot <end>',
  '<start> टॉमला भरपूर काही माहिती आहे <end>'],
 ['<start> tom likes chess <end>', '<start> टॉमला बुद्धिबळ आवडतो <end>'],
 ['<start> tom likes games <end>', '<start> टॉमला खेळ आवडतात <end>']]

In [19]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [21]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [23]:
def load_dataset(pairs, num_examples):
    # pairs => already created cleaned input, output pairs

    # index language using the class defined above    
    inp_lang = LanguageIndex(en for en, ma in pairs)
    targ_lang = LanguageIndex(ma for en, ma in pairs)
    
    # Vectorize the input and target languages
    
    # English sentences
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in pairs]
    
    # Marathi sentences
    target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [24]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(lines))

In [25]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(36675, 36675, 4076, 4076)

In [26]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [48]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [49]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [50]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [51]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [52]:
optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [53]:
checkpoint_dir = '/content/gdrive/My Drive/Colab Notebooks/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [55]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 1.4520
Epoch 1 Batch 100 Loss 0.9083
Epoch 1 Batch 200 Loss 0.9023
Epoch 1 Batch 300 Loss 0.7856
Epoch 1 Batch 400 Loss 0.8808
Epoch 1 Batch 500 Loss 0.7301
Epoch 1 Loss 0.8636
Time taken for 1 epoch 352.0294888019562 sec

Epoch 2 Batch 0 Loss 0.7931
Epoch 2 Batch 100 Loss 0.7269
Epoch 2 Batch 200 Loss 0.6014
Epoch 2 Batch 300 Loss 0.6770
Epoch 2 Batch 400 Loss 0.6790
Epoch 2 Batch 500 Loss 0.6057
Epoch 2 Loss 0.6397
Time taken for 1 epoch 356.39378023147583 sec

Epoch 3 Batch 0 Loss 0.5456
Epoch 3 Batch 100 Loss 0.5530
Epoch 3 Batch 200 Loss 0.5224
Epoch 3 Batch 300 Loss 0.4194
Epoch 3 Batch 400 Loss 0.4959
Epoch 3 Batch 500 Loss 0.4085
Epoch 3 Loss 0.4668
Time taken for 1 epoch 355.61329984664917 sec

Epoch 4 Batch 0 Loss 0.3371
Epoch 4 Batch 100 Loss 0.3391
Epoch 4 Batch 200 Loss 0.3707
Epoch 4 Batch 300 Loss 0.3119
Epoch 4 Batch 400 Loss 0.3246
Epoch 4 Batch 500 Loss 0.3717
Epoch 4 Loss 0.3339
Time taken for 1 epoch 355.25361347198486 sec

Epoch 5 Batch 0 Loss 

In [56]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [57]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    sentence = sentence[:-1]
    
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot


In [58]:
def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))
    print('Predicted translation: {}'.format(result[:-6]))
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]
    
    # use plotly to generate the heat map
    trace = go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='Reds')
    data=[trace]
    iplot(data)

In [60]:
predict_random_val_sentence()

Input: come quick
Predicted translation: लवकर ये 
Actual translation: लवकर या


In [62]:
predict_random_val_sentence()

Input: what languages do they speak in canada
Predicted translation: कॅनडामध्ये ते कोणत्या भाषा बोलतात 
Actual translation: कॅनडामध्ये कोणत्या भाषा बोलल्या जातात
